In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload

In [3]:
!git clone "https://github.com/RuslanZaripov/rhtr.git"

Cloning into 'rhtr'...
remote: Enumerating objects: 808, done.
remote: Counting objects: 100% (295/295), done.
remote: Compressing objects: 100% (203/203), done.
remote: Total 808 (delta 164), reused 206 (delta 86), pack-reused 513
Receiving objects: 100% (808/808), 53.92 MiB | 34.21 MiB/s, done.
Resolving deltas: 100% (507/507), done.


In [4]:
%cd /kaggle/working/rhtr

/kaggle/working/rhtr


In [5]:
import os
workdir = os.getcwd()
workdir 

'/kaggle/working/rhtr'

In [6]:
data_dir = f'{workdir}/data'
data_dir

'/kaggle/working/rhtr/data'

In [9]:
!mkdir {data_dir}

mkdir: cannot create directory '/kaggle/working/rhtr/data': File exists


In [10]:
!mkdir /kaggle/working/rhtr/models/
!mkdir /kaggle/working/rhtr/models/segmentation

In [11]:
!pip install transformers onnxruntime onnx imutils jiwer > /dev/null

In [12]:
!cp /kaggle/input/linknet-12-2/linknet_12_2.onnx /kaggle/working/rhtr/models/segmentation/linknet_12_2.onnx

In [13]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import os

def download_model(model_path, model_name):
    """Download a Hugging Face model and processor to the specified directory"""
    # Check if the directory already exists
    if not os.path.exists(model_path):
        # Create the directory
        os.makedirs(model_path)

    processor = TrOCRProcessor.from_pretrained(model_name)
    model = VisionEncoderDecoderModel.from_pretrained(model_name)

    # Save the model and processor to the specified directory
    model.save_pretrained(model_path)
    processor.save_pretrained(model_path)


download_model('/kaggle/working/rhtr/models/tr_ocr', 'zarus03/rhtr')

2024-05-30 13:56:28.366437: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-30 13:56:28.366543: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-30 13:56:28.529855: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.67k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 64}
Removed shared tensor {'decoder.output_projection.weight'} while saving. This should be OK, but check by verifying that you don't receive any warning while reloading


In [14]:
import src.pipeline.pipeline_predictor

predictor = src.pipeline.pipeline_predictor.PipelinePredictor(
    config_path='src/pipeline/scripts/pipeline_config.json'
)

WordSegmentation input {'model_name': 'UNet', 'model_path': 'models/segmentation/linknet_12_2.onnx', 'config_path': 'src/segmentation/configs/train_config.json'}
cwd='/kaggle/working/rhtr' config_path='src/segmentation/configs/train_config.json'
OpticalCharacterRecognition input {'model_name': 'TrOCR', 'model_path': 'models/tr_ocr/', 'ocr_classes': ['handwritten_text_shrinked_mask1']}


In [33]:
import cv2

def process(file_path):
    image = cv2.imread(file_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    try:
        rotated_image, data = predictor.predict(image)
    except Exception as e:
        print(f"Wasn't processed {file_path=}")
        
    filtered_words = list(filter(
        lambda prediction: prediction['class_name'] in predictor.get_prediction_classes(),
        data['predictions']
    ))
    
    word_count = len(filtered_words)
    print(f"{word_count=}")

    # # Sorting by word_idx
    sorted_predictions = sorted(
        filtered_words,
        key=lambda prediction: prediction['word_idx'] if 'word_idx' in prediction.keys() else word_count)

    words_string = ' '.join(prediction['text'] for prediction in sorted_predictions)
    
    return words_string

In [38]:
ocr_text = process('/kaggle/input/hwr200/HWR200/images/original0_LightPhoto_1.jpg')
ocr_text

image.shape=(3818, 3024, 3)
Function __call__ Took 13.2577 seconds
Function __call__ Took 22.0328 seconds


/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


word_count=132


'Что главное 8 жизни. семья Что гла он а 8 тиз- имен работа ни . семья ими работа - бота проблем ма, над которой рассуждает Д. Ю. Бе- ли ченно, Писа тель для полного расиры тся поста бленного боироса выбирает форму письма другу, С роде котор ому герой откровенно ряет 8се обои разл об имения, Мужчина, серьезно за солнения, думы бается, что для него ба жнее работа или силья, В результат те - они выбирает се мью. Д. Ю. Бели гемно, сам от ещ шестер бы рад такому де- ре- тей, несомненно, автора шению героя. Позиция проявляется гетко б предложении 22: 1, Семья - она важнее. Я, раз- делею тому зрения автора. семья- это, пожалуй, то единств вечное место, где тебя боепринимают томим, какой та есть, р радуются тбоим успеха м. и неголятся вместе'

In [39]:
gt_text = open('/kaggle/input/hwr200/HWR200/gts/original0_LightPhoto_1.txt').read()
gt_text

'Что главное в жизни: семья или работа? Что главное в жизни: семья или работа–вот проблема, над которой рассуждает Д.\xa0Ю.\xa0Беличенко. Писатель для полного раскрытия поставленного вопроса выбирает форму письма другу, которому герой откровенно доверяет все свои размышления, сомнения. Мужчина, серьезно задумывается, что для него важнее: работа или семья. В результате он выбирает последнее. Д. Ю. Беличенко, сам отец шестерых детей, несомненно, рад такому решению героя. Позиция автора проявляется четко в предложении 22: «Семья — она важнее». Я разделяю точку зрения автора: семья — это, пожалуй, то единственное место, где тебя воспринимают таким, какой ты есть, радуются твоим успехам и печалятся вместе'

In [40]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


def calculate_cosine_distance(text1, text2):
    # Tokenize the texts
    texts = [text1, text2]

    # texts = list(map(preprocess_text, texts))

    vectorizer = CountVectorizer(encoding='latin-1', binary=False).fit(texts)
    vectors = vectorizer.transform(texts)

    # Calculate cosine similarity
    cosine_sim = cosine_similarity(vectors)

    # The cosine similarity matrix will have cosine similarity values for all pairs of texts
    # Since we only have two texts, we return the value at (0, 1) or (1, 0)
    return cosine_sim[0][1]

In [41]:
distance = calculate_cosine_distance("This is the first text.", "text")
print("Cosine distance:", distance)

Cosine distance: 0.4472135954999579


In [43]:
from evaluate import load
wer = load("wer")
cer = load("cer")

In [44]:
cousine_score = calculate_cosine_distance(ocr_text, gt_text)
wer_score = wer.compute(predictions=[ocr_text], references=[gt_text])
cer_score = cer.compute(predictions=[ocr_text], references=[gt_text])
print(f"{cousine_score=} {wer_score=} {cer_score=}")

cousine_score=0.6458489066525221 wer_score=0.89 cer_score=0.2621082621082621
